In [1]:
import sys
from pathlib import Path
from importlib import reload

project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))

import pandas as pd
import src.model
import src.trainer
import src.visualization

reload(src.model)
reload(src.trainer)
reload(src.visualization)

from src.model import Decoder
from src.trainer import TimeSeriesDataset, TanaForecastTrainer

train_df = pd.read_csv(project_root / 'src' / 'datasets' / 'delhi' / 'DailyDelhiClimateTrain.csv')
test_df = pd.read_csv(project_root / 'src' / 'datasets' / 'delhi' / 'DailyDelhiClimateTest.csv')

train_df.head()

,date,meantemp,humidity,wind_speed,meanpressure
0,2013-01-01,10.000000,84.500000,0.000000,1015.666667
1,2013-01-02,7.400000,92.000000,2.980000,1017.800000
2,2013-01-03,7.166667,87.000000,4.633333,1018.666667
3,2013-01-04,8.666667,71.333333,1.233333,1017.166667
4,2013-01-05,6.000000,86.833333,3.700000,1016.500000


In [2]:
import torch

feature_cols = ['meantemp', 'humidity', 'wind_speed', 'meanpressure']
target_cols = ['meantemp']

train_dataset = TimeSeriesDataset(
    df=train_df,
    context_window=90,
    prediction_length=7,
    feature_columns=feature_cols,
    target_columns=target_cols,
    stride=1,
    normalize=True
)

val_dataset = TimeSeriesDataset(
    df=test_df,
    context_window=90,
    prediction_length=7,
    feature_columns=feature_cols,
    target_columns=target_cols,
    stride=1,
    normalize=True
)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Val dataset size: {len(val_dataset)}")

context, target = train_dataset[0]
print(f"Context shape: {context.shape}")
print(f"Target shape: {target.shape}")


Train dataset size: 1366
Val dataset size: 18
Context shape: torch.Size([4, 90])
Target shape: torch.Size([1, 7])


In [3]:
model = Decoder(
    context_window=90,
    prediction_length=7,
    d_model=len(feature_cols)
)

trainer = TanaForecastTrainer(
    model=model,
    train_dataset=train_dataset,
    val_dataset=val_dataset,
    batch_size=64,
    learning_rate=1e-3,
    num_epochs=200,
    checkpoint_dir=str(project_root / 'checkpoints' / 'delhi'),
    early_stopping_patience=-1
)

history = trainer.train()


Training on cpu
Total epochs: 200
Batch size: 64
Train batches: 22
Val batches: 1
------------------------------------------------------------
Epoch 1/200 | Train Loss: 0.388205 | Val Loss: 0.811592 | LR: 1.00e-03 | Time: 0.12s
  → New best model saved (Val Loss: 0.811592)
Epoch 2/200 | Train Loss: 0.153669 | Val Loss: 0.450639 | LR: 1.00e-03 | Time: 0.11s
  → New best model saved (Val Loss: 0.450639)
Epoch 3/200 | Train Loss: 0.122598 | Val Loss: 0.547513 | LR: 9.99e-04 | Time: 0.11s
Epoch 4/200 | Train Loss: 0.112774 | Val Loss: 0.430502 | LR: 9.99e-04 | Time: 0.11s
  → New best model saved (Val Loss: 0.430502)
Epoch 5/200 | Train Loss: 0.107850 | Val Loss: 0.377688 | LR: 9.98e-04 | Time: 0.10s
  → New best model saved (Val Loss: 0.377688)
Epoch 6/200 | Train Loss: 0.104150 | Val Loss: 0.392662 | LR: 9.98e-04 | Time: 0.10s
Epoch 7/200 | Train Loss: 0.093634 | Val Loss: 0.473202 | LR: 9.97e-04 | Time: 0.11s
Epoch 8/200 | Train Loss: 0.087171 | Val Loss: 0.469172 | LR: 9.96e-04 | Time:

In [4]:
from src.visualization import plot_training_history

fig = plot_training_history(history)
fig.show()

print(f"\nBest Validation Loss: {min(history['val_loss']):.6f}")
print(f"Final Train Loss: {history['train_loss'][-1]:.6f}")



Best Validation Loss: 0.321824
Final Train Loss: 0.022012


In [5]:
from src.visualization import plot_forecast, compute_metrics, print_metrics

context, target = val_dataset[10]
prediction = trainer.predict(context.unsqueeze(0)).squeeze(0)

fig = plot_forecast(
    context=context,
    target=target,
    prediction=prediction,
    dataset=val_dataset,
    title='Delhi Temperature Forecast (7-day ahead)',
    feature_idx=0,
    feature_name='Temperature (°C)'
)
fig.show()

target_denorm = val_dataset.denormalize(target, is_target=True)
prediction_denorm = val_dataset.denormalize(prediction, is_target=True)

metrics = compute_metrics(target_denorm[0], prediction_denorm[0])
print_metrics(metrics, unit="°C")


MSE:  31.4318°C
MAE:  5.4929°C
RMSE: 5.6064°C


In [6]:
prediction_denorm

tensor([24.9991, 25.0129, 25.5508, 26.1536, 26.7350, 27.1739, 27.2445])